# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 57 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.16215


extracting tarball to tmp_2303.16215...

 done.


Found 101 bibliographic references in tmp_2303.16215/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2303.16217


...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.16810


extracting tarball to tmp_2303.16810...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.16849


extracting tarball to tmp_2303.16849...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.16215-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.16215) | **The Circumgalactic Medium of Milky Way-like Galaxies in the TNG50  Simulation -- II: Cold, Dense Gas Clouds and High-Velocity Cloud Analogs**  |
|| R. Ramesh, D. Nelson, <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-03-30*|
|*Comments*| *Accepted for publication (MNRAS). Part of a set of papers based on TNG50 MW/M31-like galaxies. Additional visuals and data products at www.tng-project.org/ramesh23b*|
|**Abstract**| We use the TNG50 simulation of the IllustrisTNG project to study cold, dense clouds of gas in the circumgalactic media (CGM) of Milky Way-like galaxies. We find that their CGM is typically filled with of order one hundred (thousand) reasonably (marginally) resolved clouds, possible analogs of high-velocity clouds (HVCs). There is a large variation in cloud abundance from galaxy to galaxy, and the physical properties of clouds that we explore -- mass, size, metallicity, pressure, and kinematics -- are also diverse. We quantify the distributions of cloud properties and cloud-background contrasts, providing cosmological inputs for idealized simulations. Clouds characteristically have sub-solar metallicities, diverse shapes, small overdensities ($\chi = n_{\rm cold} / n_{\rm hot} \lesssim 10$), are mostly inflowing, and have sub-virial rotation. At TNG50 resolution, resolved clouds have median masses of $\sim 10^6\,\rm{M_\odot}$ and sizes of $\sim 10$ kpc. Larger clouds are well converged numerically, while the abundance of the smallest clouds increases with resolution, as expected. In TNG50 MW-like haloes, clouds are slightly (severely) under-pressurised relative to their surroundings with respect to total (thermal) pressure, implying that magnetic fields may be important. Clouds are not distributed uniformly throughout the CGM, but are clustered around other clouds, often near baryon-rich satellite galaxies. This suggests that at least some clouds originate from satellites, via direct ram-pressure stripping or otherwise. Finally, we compare with observations of intermediate and high velocity clouds from the real Milky Way halo. TNG50 shows a similar cloud velocity distribution as observations, and predicts a significant population of currently difficult-to-detect low velocity clouds. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.16217-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.16217) | **Milky Way and Andromeda analogs from the TNG50 simulation**  |
|| <mark>A. Pillepich</mark>, et al. |
|*Appeared on*| *2023-03-30*|
|*Comments*| *Submitted to MNRAS. Feedback from the community encouraged. This is also a data-release paper: see visuals and documentation at this https URL See other TNG50 MW/M31-based papers also on astro-ph today: Ramesh+ and Sotillo-Ramos+*|
|**Abstract**| We present the properties of Milky Way- and Andromeda-like (MW/M31-like) galaxies simulated within TNG50, the highest-resolution run of the IllustrisTNG suite of $\Lambda$CDM magneto-hydrodynamical simulations. We introduce our fiducial selection for MW/M31 analogs, which we propose for direct usage as well as for reference in future analyses. TNG50 contains 198 MW/M31 analogs, i.e. galaxies with stellar disky morphology, with a stellar mass in the range of $M_* = 10^{10.5 - 11.2}$ Msun, and within a MW-like Mpc-scale environment at z=0. These are resolved with baryonic (dark matter) mass resolution of $8.5\times10^4$ Msun ($4.5\times10^5$ Msun) and $\sim150$ pc of average spatial resolution in the star-forming regions: we therefore expand by many factors (2 orders of magnitude) the sample size of cosmologically-simulated analogs with similar ($\times 10$ better) numerical resolution. The majority of TNG50 MW/M31 analogs at $z=0$ exhibit a bar, 60 per cent are star-forming, the sample includes 3 Local Group (LG)-like systems, and a number of galaxies host one or more satellites as massive as e.g. the Magellanic Clouds. Even within such a relatively narrow selection, TNG50 reveals a great diversity in galaxy and halo properties, as well as in past histories. Within the TNG50 sample, it is possible to identify several simulated galaxies whose integral and structural properties are consistent, one or more at a time, with those measured for the Galaxy and Andromeda. With this paper, we document and release a series of broadly applicable data products that build upon the IllustrisTNG public release and aim to facilitate easy access and analysis by public users. These include datacubes across snapshots ($0 \le z \le 7$) for each TNG50 MW/M31-like galaxy, and a series of value-added catalogs that will be continually expanded to provide a convenient and up to date community resource. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.16228-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.16228) | **Disk flaring with TNG50: diversity across Milky Way and M31 analogs**  |
|| D. Sotillo-Ramos, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-03-30*|
|*Comments*| *Submitted to MNRAS. Main figures: 5, 13. See presentation and data release of TNG50 MW/M31 analogs by Pillepich et al. and see also Ramesh et al. on astro-ph today*|
|**Abstract**| We use the sample of 198 Milky Way (MW) and Andromeda (M31) analogs from TNG50 to quantify the level of disk flaring predicted by a modern, high-resolution cosmological hydrodynamical simulation. Disk flaring refers to the increase of vertical stellar disk height with galactocentric distance. The TNG50 galaxies are selected to have stellar disky morphology, a stellar mass in the range of $M_* = 10^{10.5 - 11.2}~\rm{M_{\odot}}$, and a MW-like Mpc-scale environment at $z=0$. The stellar disks of such TNG50 MW/M31 analogs exhibit a wide diversity of structural properties, including a number of galaxies with disk scalelength and thin and thick disk scaleheights that are comparable to those measured or inferred for the Galaxy and Andromeda. With one set of physical ingredients, TNG50 returns a large variety of flaring flavours and amounts, also for mono-age stellar populations. With this paper, we hence propose a non-parametric characterization of flaring. The typical MW/M31 analogs exhibit disk scaleheights that are $1.5-2$ times larger in the outer than in the inner regions of the disk for both old and young stellar populations, but with a large galaxy-to-galaxy variation. Which stellar population flares more, and by how much, also varies from galaxy to galaxy. TNG50 de facto brackets existing observational constraints for the Galaxy and all previous numerical findings. A link between the amount of flaring and the $z=0$ global galaxy structural properties or merger history is complex. However, a connection between the scaleheights and the local stellar vertical kinematics and gravitational potential is clearly in place. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.16232-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.16232) | **Global dynamic scaling relations of HI-rich ultra-diffuse galaxies**  |
|| H.-J. Hu, et al. -- incl., <mark>H.-X. Zhang</mark> |
|*Appeared on*| *2023-03-30*|
|*Comments*| *12 pages, 9 figures, 1 table, accepted for publication in ApJL*|
|**Abstract**| The baryonic Tully-Fisher relation (BTFR), which connects the baryonic mass of galaxies with their circular velocities, has been validated across a wide range of galaxies, from dwarf galaxies to massive galaxies. Recent studies have found that several ultra-diffuse galaxies (UDGs) deviate significantly from the BTFR, indicating a galaxy population with abnormal dynamical properties. However, such studies were still confined within a small sample size. In this study, we used the 100% complete Arecibo Legacy Fast Arecibo L-band Feed Array (ALFALFA) to investigate the BTFR of 88 HI-rich UDGs (HUDGs), which is the largest UDG sample with dynamical information. We found that the HUDGs form a continuous distribution in the BTFR diagram, with high-velocity galaxies consistent with normal dwarf galaxies at 1 $\sigma$ level, and low-velocity galaxies deviating from the BTFR, in line with that reported in the literature. We point out that the observed deviation may be subject to various selection effects or systemic biases. Nevertheless, we found that the significance of the deviation of HUDGs from the BTFR and TFR are different, i.e., they either deviate from the BTFR or from the TFR. Our result indicates that a high-gas fraction may play an important role in explaining the deviation of HUDGs from BTFR. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.16775-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.16775) | **Fast radio burst energy function in the presence of $\rm DM_{host}$  variation**  |
|| Y. Li, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-03-30*|
|*Comments*| *8 pages, 5 figures*|
|**Abstract**| Fast radio bursts (FRBs) have been found in great numbers but the physical mechanism of these sources is still a mystery. The redshift evolutions of the FRB energy distribution function and the volumetric rate shed light on revealing the origin of the FRBs. However, such estimations rely on the dispersion measurement (DM)-redshift ($z$) relationship. A few of FRBs detected recently show large excess DM beyond the expectation from the cosmological and Milky Way contributions, which indicates large spread of DM from their host galaxies. In this work, we adopt the lognormal distributed $\rm DM_{host}$ model and estimate the energy function using the non-repeating FRBs selected from the Canadian Hydrogen Intensity Mapping Experiment (CHIME)/FRB Catalog 1. By comparing the lognormal distributed $\rm DM_{host}$ model to the constant $\rm DM_{host}$ model, the FRB energy function results are consistent within the measurement uncertainty. We also estimate the volumetric rate of the non-repeating FRBs in three different redshift bins. The volumetric rate shows that the trend is consistent with the stellar-mass density redshift evolution. Since the lognormal distributed $\rm DM_{host}$ model increases the measurement errors, the inference of FRBs tracking the stellar-mass density is nonetheless undermined. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.16810-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.16810) | **Implications of GWTC-3 on primordial black holes from vacuum bubbles**  |
|| <mark>J. He</mark>, H. Deng, Y.-S. Piao, J. Zhang |
|*Appeared on*| *2023-03-30*|
|*Comments*| **|
|**Abstract**| The population of black holes inferred from the detection of gravitational waves by the LIGO-Virgo-KAGRA collaboration has revealed interesting features in the properties of black holes in the universe. We analyze the GWTC-3 dataset assuming the detected black holes have both astrophysical and primordial origins. In particular, we consider primordial black holes forming from vacuum bubbles that nucleate during inflation, with their mass distribution described by a broken power law. We find that more than half of the events could come from primordial black hole mergers. Astrophysical black holes are mainly responsible for the peak in mass distribution at $\sim 10M_\odot$ indicated by GWTC-3; whereas primordial black holes are responsible for the massive black holes, as well as the peak at $\sim 30M_\odot$. We also discuss the implications on the primordial black hole formation mechanism and the underlying inflationary model. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.16849-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.16849) | **Dwarf Galaxy Discoveries from the KMTNet Supernova Program III. the  Milky-Way Analog NGC~2997 Group**  |
|| T. J. Fan, et al. -- incl., <mark>T. S. Li</mark> |
|*Appeared on*| *2023-03-30*|
|*Comments*| **|
|**Abstract**| We present the discovery of 48 new and the analysis of 55, including 7 previously discovered, dwarf galaxy candidates around the giant spiral galaxy NGC~2997 using deep $BVI$ images from the KMTNet Supernova Program. Their $V$-band central surface brightness and total absolute magnitudes range between 20.3--26.7 mag arcsec$^{-2}$ and --(8.02--17.69) mag, respectively, while the $I$-band effective radii are between 0.14 and 2.97 kpc. We obtain $\alpha$ $\simeq$ --1.43 $\pm$ 0.02 for the faint-end slope of their luminosity function, comparable to previously measured values but shallower than theoretical predictions based on $\Lambda$CDM models. The distance-independent distributions of their mass and color from the host galaxy NGC~2997 suggest that the group could be dynamically young, prior to the development of significant mass segregation or radial color gradients. The systematically bluer colors of the brighter candidates than the fainter ones indicate higher star formation activities in brighter members. We suggest that the higher-mass dwarf galaxies in the group have maintained star-formation activities by effectively retaining gas content, while environmental quenching is only effective for the lower-mass galaxies. The interpretation of early evolutionary stage of this group is also consistent with the overall morphological distribution of the dwarf galaxy candidates showing a lack of morphologically evolved candidates but a plethora of irregularly shaped ones. Our detection rate of dwarf galaxy candidates in the NGC~2997 group and their inferred star formation activities are largely comparable to those found in Milky Way analog systems from the SAGA survey within the magnitude limit M$_{V}$ $\lesssim$ --13 mag, as well as those found in the ELVES survey. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.16439-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.16439) | **NGC 6302: The Tempestuous Life of a Butterfly**  |
|| B. Balick, et al. -- incl., <mark>A. Frank</mark> |
|*Appeared on*| *2023-03-30*|
|*Comments*| *17 pages, 3 figures, 1 table*|
|**Abstract**| NGC 6302 (The ''Butterfly Nebula'') is an extremely energetic bipolar nebula whose central star is among the most massive, hottest, and presumably rapidly evolving of all central stars of planetary nebulae. Our proper-motion study of NGC 6302, based on excellent HST WFC3 images spanning 11 yr, has uncovered at least four different pairs of expanding internal lobes that were ejected at various times over the past two millennia at speeds ranging from 10 to 600 km s^-1. In addition, we find a pair of off-axis flows in constant motion at 760 +/- 100 km s^-1 within which bright [Fe II] feathers are conspicuous. Combining our results with those previously published, we find that the ensemble of flows has an ionized mass > 0.1 M_sun. The kinetic energy of the ensemble, 10^46 - 10^48 ergs, lies at the upper end of gravity-powered processes such as stellar mergers or mass accretion and is too large to be explained by stellar radiation pressure or convective ejections. The structure and dynamics of the Butterfly Nebula suggests that its central engine has had a remarkable history, and the highly unusual patterns of growth within its wings challenge our current understanding of late stellar mass ejection. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.16215.md
    + _build/html/tmp_2303.16215/./figures/CloudTemperaturePdf.png
    + _build/html/tmp_2303.16215/./figures/CloudDensityPdf.png
    + _build/html/tmp_2303.16215/./figures/CloudMetallicityPdf.png
    + _build/html/tmp_2303.16215/./figures/CloudPresRatPdf.png
    + _build/html/tmp_2303.16215/./figures/CloudRadVelPdf.png
    + _build/html/tmp_2303.16215/./figures/CloudRotVelNormPdf.png
    + _build/html/tmp_2303.16215/./figures/CloudTempBackConHist.png
    + _build/html/tmp_2303.16215/./figures/CloudDensBackConHist.png
    + _build/html/tmp_2303.16215/./figures/CloudTotPresBackConHist.png
    + _build/html/tmp_2303.16215/./figures/CloudThmPresBackConHist.png
    + _build/html/tmp_2303.16215/./figures/CloudMetalBackConHist.png
    + _build/html/tmp_2303.16215/./figures/CloudRadVelBackConHist.png
    + _build/html/tmp_2303.16215/./figures/bhMassCbar.png
    + _build/html/tmp_2303.16215/./figures/CloudMassPdf.png
    + _build/html/tmp_2303.16215/./fig

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\}{MSUN}$
$\newcommand{\}{RVIR}$
$\newcommand{\}{RTC}$
$\newcommand{\}{MTC}$
$\newcommand{\}{MSTC}$
$\newcommand{\}{RFC}$
$\newcommand{\}{MFC}$
$\newcommand{\}{ROUT}$
$\newcommand{\}{RHALF}$
$\newcommand{\}{MVIR}$
$\newcommand{\}{MHALO}$
$\newcommand{\}{MSH}$
$\newcommand{\}{MS}$
$\newcommand{\}{AS}$
$\newcommand{\}{ADM}$
$\newcommand{\}{etal}$</div>



<div id="title">

# The Circumgalactic Medium of Milky Way-like Galaxies in the TNG50 Simulation -- II: Cold, Dense Gas Clouds and High-Velocity Cloud Analogs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.16215-b31b1b.svg)](https://arxiv.org/abs/2303.16215)<mark>Appeared on: 2023-03-30</mark> -  _Accepted for publication (MNRAS). Part of a set of papers based on TNG50 MW/M31-like galaxies. Additional visuals and data products at www.tng-project.org/ramesh23b_

</div>
<div id="authors">

R. Ramesh, D. Nelson, <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** We use the TNG50 simulation of the IllustrisTNG project to study cold, dense clouds of gas in the circumgalactic media (CGM) of Milky Way-like galaxies. We find that their CGM is typically filled with of order one hundred (thousand) reasonably (marginally) resolved clouds, possible analogs of high-velocity clouds (HVCs). There is a large variation in cloud abundance from galaxy to galaxy, and the physical properties of clouds that we explore -- mass, size, metallicity, pressure, and kinematics -- are also diverse. We quantify the distributions of cloud properties and cloud-background contrasts, providing cosmological inputs for idealized simulations. Clouds characteristically have sub-solar metallicities, diverse shapes, small overdensities ( $\chi = n_{\rm cold} / n_{\rm hot} \lesssim 10$ ), are mostly inflowing, and have sub-virial rotation. At TNG50 resolution, resolved clouds have median masses of $\sim$ $10^6 \rm{M_\odot}$ and sizes of $\sim$ $ 10$ kpc. Larger clouds are well converged numerically, while the abundance of the smallest clouds increases with resolution, as expected. In TNG50 MW-like haloes, clouds are slightly (severely) under-pressurised relative to their surroundings with respect to total (thermal) pressure, implying that magnetic fields may be important. Clouds are not distributed uniformly throughout the CGM, but are clustered around other clouds, often near baryon-rich satellite galaxies. This suggests that at least some clouds originate from satellites, via direct ram-pressure stripping or otherwise. Finally, we compare with observations of intermediate and high velocity clouds from the real Milky Way halo. TNG50 shows a similar cloud velocity distribution as observations, and predicts a significant population of currently difficult-to-detect low velocity clouds.

</div>

<div id="div_fig1">

<img src="tmp_2303.16215/./figures/CloudTemperaturePdf.png" alt="Fig5.1" width="16%"/><img src="tmp_2303.16215/./figures/CloudDensityPdf.png" alt="Fig5.2" width="16%"/><img src="tmp_2303.16215/./figures/CloudMetallicityPdf.png" alt="Fig5.3" width="16%"/><img src="tmp_2303.16215/./figures/CloudPresRatPdf.png" alt="Fig5.4" width="16%"/><img src="tmp_2303.16215/./figures/CloudRadVelPdf.png" alt="Fig5.5" width="16%"/><img src="tmp_2303.16215/./figures/CloudRotVelNormPdf.png" alt="Fig5.6" width="16%"/>

**Figure 5. -** Distributions of physical properties of clouds in the CGM of TNG50 MW-like galaxies. In each panel, we show the median behaviour across the full sample of galaxies, split into three regions of the halo: inner halo (solid curves; $0.15 < r/\rm{R_{200,c}} \leq 0.4$), central halo (dashed curves; $0.4 < r/\rm{R_{200,c}} \leq 0.7$) and outer halo (dotted curves; $0.7 < r/\rm{R_{200,c}} \leq 1.0$). The two upper panels show the distributions of temperature (left) and density (right): clouds closer to the centre of the halo are cooler and denser than those farther away. The centre-left and right-panels show the distribution of cloud metallicity and magnetic to thermal pressure ratio ($\beta^{-1}$), respectively: while clouds throughout the halo  mainly possess sub-solar metallicity and pressure ratios close to unity, clouds in the inner halo are the most likely to be highly enriched and dominated by magentic pressure. The bottom-left and -right panels show distributions of the radial velocity (in km/s) and rotational velocity (normalised by the virial velocity), respectively: a smaller fraction of clouds in the inner halo have dominant rotational motion and a greater fraction are either strongly inflowing or outflowing. Finally, the upper two rows also compare values derived in two ways: the mean value across the gas within each cloud (black; as in the lower panels), and the 90$^{\rm{th}}$ percentile value of the gas properties within each cloud (red). The insets show the relation between these two values, demonstrating that TNG50 clouds exhibit inner inhomogeneities in their metal content and temperature-, density- and pressure-structures. (*fig:cloudBasicProp2*)

</div>
<div id="div_fig2">

<img src="tmp_2303.16215/./figures/CloudTempBackConHist.png" alt="Fig8.1" width="16%"/><img src="tmp_2303.16215/./figures/CloudDensBackConHist.png" alt="Fig8.2" width="16%"/><img src="tmp_2303.16215/./figures/CloudTotPresBackConHist.png" alt="Fig8.3" width="16%"/><img src="tmp_2303.16215/./figures/CloudThmPresBackConHist.png" alt="Fig8.4" width="16%"/><img src="tmp_2303.16215/./figures/CloudMetalBackConHist.png" alt="Fig8.5" width="16%"/><img src="tmp_2303.16215/./figures/CloudRadVelBackConHist.png" alt="Fig8.6" width="16%"/>

**Figure 8. -** Distributions of contrast in properties of TNG50 MW-like clouds with respect to their intermediate and background layers, i.e. with respect to the local, ambient CGM gas. In each panel, distributions of contrasts corresponding to the former are shown in black, while the latter are shown in red. We show median PDFs for three distance regimes: inner halo (solid curves; $0.15 < r/\rm{R_{200,c}} \leq 0.4$), central halo (dashed; $0.4 < r/\rm{R_{200,c}} \leq 0.7$) and outer halo (dotted; $0.7 < r/\rm{R_{200,c}} \leq 1.0$). Contrasts of temeprature are shown in the top-left panel, density in the top-right, total pressure in the centre-left, thermal pressure in the centre-right, metallicity in the bottom-left, and radial velocity in the bottom-right. In all cases, contrasts are more enhanced with respect to the background as compared to the intermediate layer, both in terms of the median, and the width of the distributions. (*fig:cloudPropVsBackPropHist*)

</div>
<div id="div_fig3">

<img src="tmp_2303.16215/./figures/bhMassCbar.png" alt="Fig4.1" width="20%"/><img src="tmp_2303.16215/./figures/CloudMassPdf.png" alt="Fig4.2" width="20%"/><img src="tmp_2303.16215/./figures/CloudSizePdf.png" alt="Fig4.3" width="20%"/><img src="tmp_2303.16215/./figures/CloudSizeVsCloudMass.png" alt="Fig4.4" width="20%"/><img src="tmp_2303.16215/./figures/CloudSizeVsCloudDist_Num.png" alt="Fig4.5" width="20%"/>

**Figure 4. -** Properties of the clouds in the CGM of TNG50 MW-like galaxies: the top-left and top-right panels show the cloud mass and cloud size distributions, respectively. In each panel, the thin curves correspond to individual haloes, color coded by central SMBH mass. Solid black curves show the median PDF across the sample, and purple and yellow curves correspond to median distributions of the two extreme octile regions, splitting by SMBH mass. The top-left panel includes an inset where the mass function of clouds is compared to lower resolution versions (TNG50-2 and TNG50-3) of TNG50 (i.e. TNG50-1), demonstrating good resolution convergence above the resolution limit threshold of each simulation. The top-right panel shows an inset corresponding to the relation between the major-to-minor axis ratio ($q$) and cloud size. In the bottom-left panel, we show the cloud mass-size relation, with the background colored by the average number of gas cells per gas cloud, and the bottom right panel shows the cloud distance-size relation, with the colored instead showing the total number of clouds in each given pixel. The most abundant clouds have masses of $\sim 10^{6}  \rm{M_\odot}$ and sizes of $\sim 10$ kpc. Lower mass clouds, with smaller sizes, tend to reside at smaller galactocentric distances. (*fig:cloudBasicProp1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2303.16215"></div>

, together with the escape velocity from a point mass of $10^{12} \MSUN$ for the MW and $10^{12.4} \MSUN$ for M31 (dotted curves). (*fig:sats*)

</div>
<div id="div_fig2">

<img src="tmp_2303.16217/./figures/fig_L35n2160TNG_StellarMasses_in_r30ckpc_SMBH_True_Masses.png" alt="Fig19.1" width="25%"/><img src="tmp_2303.16217/./figures/fig_L35n2160TNG_StellarMasses_in_r30ckpc_EddingtonRatio.png" alt="Fig19.2" width="25%"/><img src="tmp_2303.16217/./figures/fig_L35n2160TNG_B2T_PB2T_SubhaloBHMass_BulgesAndPseudoBulges.png" alt="Fig19.3" width="25%"/><img src="tmp_2303.16217/./figures/fig_L35n2160TNG_SFR_MsunPerYrs_in_r30pkpc_50Myrs_number_forBars_WithComp.png" alt="Fig19.4" width="25%"/>

**Figure 19. -** ** The fundamental characteristics of the central regions of TNG50 MW/M31 galaxies.** In the top panels, we showcase the SMBH populations of MW/M31 analogs (blue circles), including their mass (left) and their gas accretion rates, i.e. Eddington ratios (right). We indicate with circles (crosses) the SMBHs that are in the so-called thermal (kinetic) feedback mode at $z=0$. In the bottom left, we quantify the bulge-to-total stellar mass ratio of the sample, contrasted to the stellar mass ratio of pseudo-bulges and connected to the mass of the central SMBHs (colors). The bottom right panel shows that at least 2/3 of TNG50 MW/M31-like galaxies exhibit a bar. (*fig:center*)

</div>
<div id="div_fig3">

<img src="tmp_2303.16217/./figures/fig_DSR_Mass_Rd_TNG50.png" alt="Fig17.1" width="33%"/><img src="tmp_2303.16217/./figures/fig_DSR_Mass_heights_8kpc_TNG50.png" alt="Fig17.2" width="33%"/><img src="tmp_2303.16217/./figures/fig_DSR_Mass_heights_3_7rd_TNG50.png" alt="Fig17.3" width="33%"/>

**Figure 17. -** **The structural properties of the stellar disks of TNG50 MW/M31 galaxies.** We showcase disk scalelengths (top panel) and disk scaleheights (left and right bottom panels) as a function of the galaxy stellar mass, for the 198 MW/M31-like galaxies at $z=0$.
The scaleheights are evaluated at cylindrical shells located from the center at $7-9$ kpc (bottom left) and at $(2.7-4.7)\times$ the scalelength (bottom right, respectively): see text details.
The magenta and orange rectangles in the top panel represent observational estimates of the scalelength for the Galaxy \citep[1.7-2.9 kpc,][]{Hammer.2007,Juric.2008,Bovy.2013} and for Andromeda \citep[4.8--6.8 kpc,][]{Worthey.2005,Barmby.2006,Hammer.2007}, respectively.
In the bottom panels we show the scaleheights of the geometrically thin (empty symbols) and geometrically thick (full symbols) stellar disks; two sets of rectangles indicate the values of thin and thick disk inferred for the Milky Way \citep[magenta:][]{Gilmore.1983, Siegel.2002, Juric.2008, Rix.2013, Bland-Hawthorn.2016} and for Andromeda \citep[orange:][]{Collins.2011}. To guide the eye the thin and thick disk of each galaxy are also connected with a solid line. A number of TNG50 galaxies are identified as MW and M31 analogs, according to their mass, disk length and disk height values: their SubhaloIDs are given in the text. (*fig:stellardisks*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2303.16217"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bea}{\begin{eqnarray}}$
$\newcommand{\eea}{\end{eqnarray}}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\Ms}{{\rm M}_\odot}$
$\newcommand{\}{MS}{M_{\rm stars}}$
$\newcommand{\}{RS}{R_{\rm stars,1/2}}$
$\newcommandinecolor{portlandorange}{rgb}{1.0, 0.35, 0.21}$
$\newcommand{\md}{\color{portlandorange}}$
$\newcommand{\ap}{\color{magenta}}$
$\newcommand{\apq}{\color{red}}$
$\newcommand{\ch}{\color{cyan}}$
$\newcommandinecolor{othergreen}{rgb}{0.0, 0.7, 0.0}$
$\newcommand{\dsr}{\color{othergreen}}$
$\newcommand{\dsrq}{\color{orange}}$
$\newcommand{\nf}{\color{purple}}$
$\newcommand{\nfq}{\color{violet}}$
$\title[Flaring of stellar disks in TNG50 MW/M31-like galaxies]{Disk flaring with TNG50: diversity across Milky Way and M31 analogs}$
$\author[Sotillo-Ramos et al.]{Diego Sotillo-Ramos,^{1}\thanks{E-mail: sotillo@mpia.de},$
$Martina Donnari^{1}, Annalisa Pillepich^{1}, Neige Frankel^{2}, Dylan Nelson^{3},$
$\newauthor Volker Springel^{4}, and Lars Hernquist^{5} \^{1} Max-Planck-Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany\^{2} Canadian Institute for Theoretical Astrophysics, University of Toronto, 60 St. George Street, Toronto, ON M5S 3H8, Canada\^{3} Universität Heidelberg, Zentrum für Astronomie, Institut für theoretische Astrophysik, Albert-Ueberle-Str. 2, 69120 Heidelberg, Germany\^{4} Max Planck Institut für Astrophysik, Karl-Schwarzschild-Stra\ss e 1, D-85748 Garching bei München, Germany\^{5} Center for Astrophysics | Harvard \& Smithsonian, 6P Garden St., Cambridge, MA 02138, USA$
$}$
$\begin{document}$
$\maketitle$
$\begin{abstract}$
$We use the sample of 198 Milky Way (MW) and Andromeda (M31) analogs from TNG50 to quantify the level of disk flaring predicted by a modern, high-resolution cosmological hydrodynamical simulation. Disk flaring refers to the increase of vertical stellar disk height with galactocentric distance. The TNG50 galaxies are selected to have stellar disky morphology, a stellar mass in the range of M_* = 10^{10.5 - 11.2}~\Ms, and a MW-like Mpc-scale environment at z=0.$
$The stellar disks of such TNG50 MW/M31 analogs exhibit a wide diversity of structural properties, including a number of galaxies with disk scalelength and thin and thick disk scaleheights that are comparable to those measured or inferred for the Galaxy and Andromeda.$
$With one set of physical ingredients, TNG50 returns a large variety of  flaring flavours and amounts, also for mono-age stellar populations. With this paper, we hence propose a non-parametric characterization of flaring. The typical MW/M31 analogs exhibit disk scaleheights that are 1.5-2 times larger in the outer than in the inner regions of the disk for both old and young stellar populations, but with a large galaxy-to-galaxy variation. Which stellar population flares more, and by how much, also varies from galaxy to galaxy. TNG50 de facto brackets existing observational constraints for the Galaxy and all previous numerical findings. A link between the amount of flaring and the z=0 global galaxy structural properties or merger history is complex. However, a connection between the scaleheights and the local stellar vertical kinematics and gravitational potential is clearly in place.$
$\end{abstract}$
$\begin{keywords}$
$methods: numerical --- galaxies: formation  --- Galaxy: disc --- Galaxy: evolution --- Galaxy: structure$
$\end{keywords}$
$\n\end{document}\end{eqnarray}}$
$\newcommand{\eea}{\end{eqnarray}}$
$\newcommand{\Ms}{{\rm M}_\odot}$
$\newcommand{\md}{\color{portlandorange}}$
$\newcommand{\ap}{\color{magenta}}$
$\newcommand{\apq}{\color{red}}$
$\newcommand{\ch}{\color{cyan}}$
$\newcommand{\dsr}{\color{othergreen}}$
$\newcommand{\dsrq}{\color{orange}}$
$\newcommand{\nf}{\color{purple}}$
$\newcommand{\nfq}{\color{violet}}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\mh}{M_\bullet}$
$\newcommand{\ead}{\cal E}$
$\newcommand{\mh}{M_\bullet}$
$\newcommand{\ead}{\cal E}$
$\newcommand{\}{MS}$
$\newcommand{\}{RS}$</div>



<div id="title">

# Disk flaring with TNG50: diversity across Milky Way and M31 analogs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.16228-b31b1b.svg)](https://arxiv.org/abs/2303.16228)<mark>Appeared on: 2023-03-30</mark> -  _Submitted to MNRAS. Main figures: 5, 13. See presentation and data release of TNG50 MW/M31 analogs by Pillepich et al. and see also Ramesh et al. on astro-ph today_

</div>
<div id="authors">

D. Sotillo-Ramos, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** We use the sample of 198 Milky Way (MW) and Andromeda (M31) analogs from TNG50 to quantify the level of disk flaring predicted by a modern, high-resolution cosmological hydrodynamical simulation. Disk flaring refers to the increase of vertical stellar disk height with galactocentric distance. The TNG50 galaxies are selected to have stellar disky morphology, a stellar mass in the range of $M_* = 10^{10.5 - 11.2} \Ms$ , and a MW-like Mpc-scale environment at $z=0$ .The stellar disks of such TNG50 MW/M31 analogs exhibit a wide diversity of structural properties, including a number of galaxies with disk scalelength and thin and thick disk scaleheights that are comparable to those measured or inferred for the Galaxy and Andromeda.With one set of physical ingredients, TNG50 returns a large variety of  flaring flavours and amounts, also for mono-age stellar populations. With this paper, we hence propose a non-parametric characterization of flaring. The typical MW/M31 analogs exhibit disk scaleheights that are $1.5-2$ times larger in the outer than in the inner regions of the disk for both old and young stellar populations, but with a large galaxy-to-galaxy variation. Which stellar population flares more, and by how much, also varies from galaxy to galaxy. TNG50 de facto brackets existing observational constraints for the Galaxy and all previous numerical findings. A link between the amount of flaring and the $z=0$ global galaxy structural properties or merger history is complex. However, a connection between the scaleheights and the local stellar vertical kinematics and gravitational potential is clearly in place.

</div>

<div id="div_fig1">

<img src="tmp_2303.16228/./figures/Flare_412kpc_TNG50.png" alt="Fig14.1" width="50%"/><img src="tmp_2303.16228/./figures/Flare_412kpc_TNG50_All_3Gyr.png" alt="Fig14.2" width="50%"/>

**Figure 14. -** **Flaring of TNG50 MW/M31 analogs in comparison to the results of other cosmological MW-like galaxy simulations**. We show again the flaring of old and young stars for TNG50 MW/M31 analogs in comparison to, on the left, simulations from \citet[][VINTERGATAN: thin and thick crosses, with young stellar populations of 2-3 Gyr and 3-4 Gyr, respectively]{2021Agertz}, \citet[][Latte: diamonds]{2017Ma}, and \citet[][NIHAO-UHD: plus symbols]{2020Buck}; on the right, simulations from the Auriga project \citep{2017Grand}. The magenta area denotes the stellar flaring of young stars of the Milky Way extrapolated from \citet{2019Ting}, as in Fig. \ref{fig:young_old}.
We note that in all the simulation models used for this comparison (except for Latte that uses ${\rm sech}^2$), the scaleheight of the mono-age stellar populations in the disk is evaluated from a single exponential fit. In these plots, the heights are hence measured using exponential profiles also for TNG50.
 (*fig:tng50vsothers*)

</div>
<div id="div_fig2">

<img src="tmp_2303.16228/./figures/scatter_sigmaVoverSurfDens_vs_scaleHeight_1sech2_young.png" alt="Fig10.1" width="50%"/><img src="tmp_2303.16228/./figures/scatter_sigmaVoverSurfDens_vs_scaleHeight_1sech2_old.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** **Relationship between stellar heights and stellar kinematics for TNG50 MW/M31-like galaxies**. We plot the ratio between the squared local stellar vertical velocity dispersion and the stellar surface density vs. stellar disk scaleheight, for young (left) and old (right) stellar populations. We do so at different locations within the stellar disk.
The dashed line defines the linear correspondence that is  expected for an idealized self-gravitating disk. (*fig:young_old_sigmaVSurfDens*)

</div>
<div id="div_fig3">

<img src="tmp_2303.16228/./figures/mosaic_previous_works.png" alt="Fig13" width="100%"/>

**Figure 13. -** **Stellar age distributions in the MW/M31 midplane**: vertical distance from the midplane as a function of galactocentric distance with the color code representing the mean or median stellar age. Here we compare observations from \citet{2016Ness,2017Xiang,2019Feuillet}(left), simulations from \citet{2017Ma,2018Navarro,2021Agertz}(center), and a sample of five TNG50 MW/M31 analogs (right). Barring those in the right column, the plots are replicated from the respective research studies without modification. (*fig:flaring_obs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2303.16228"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

180  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
